In [1]:
import csv
from pathlib import Path
import pandas as pd
import numpy as np

In [36]:
label_map = {
    "tumor_any": 0,
    "nonTIL_stromal": 1,
    "sTIL": 2,
    "AMBIGUOUS": 3
}

In [38]:
# plot an overlay
from PIL import Image, ImageDraw

image_folder = "./test/images"
label_folder = "./test/labels"
overlay_folder = "./test/overlay"

image_folder = Path(image_folder)
label_folder = Path(label_folder)
overlay_folder = Path(overlay_folder)

for image in image_folder.glob("*.png"):
    img = Image.open(image)
    overlay = Image.new("RGBA", img.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(overlay)
    
    cell_annotation_file = label_folder / (image.stem + ".csv")
    if not cell_annotation_file.exists():
        continue
    with open(cell_annotation_file, "r") as f:
        cells_pd = csv.reader(f)
        cells = [(int(x), int(y), int(label)) for x, y, label in cells_pd]
    
    # draw a circle around each cell with the correct color
    for cell in cells:
        x, y, label = cell
        radius = 3  # Adjust this for larger points
        if label == 0:
            draw.ellipse([x - radius, y - radius, x + radius, y + radius], fill=(255, 0, 0, 128))  # Red points
        elif label == 1:
            draw.ellipse([x - radius, y - radius, x + radius, y + radius], fill=(0, 255, 0, 128))  # Green points
        elif label == 2:
            draw.ellipse([x - radius, y - radius, x + radius, y + radius], fill=(0, 0, 255, 128))  # Blue points
        elif label == 3:
            draw.ellipse([x - radius, y - radius, x + radius, y + radius], fill=(255, 255, 0, 128)) # Yellow points
    img = Image.alpha_composite(img.convert("RGBA"), overlay)
    img.save(overlay_folder / image.name)